# 火炬内部之旅(第一部分)
> 原文地址：[A Tour of PyTorch Internals (Part I)](https://pytorch.org/2017/05/11/Internals.html) 
> 希望从翻译这篇文章开始能认真的去读一下pytorch源码

本文进度  
- [X] 简介  
- [X] 扩展 Python 解释器  
- [ ] The THPTensor Type  
- [ ] Generic Builds  
- [ ] PyTorch 封装

火炬的基本单位是张量。 这篇文章将作为我们如何在 PyTorch 中实现张量的概述，这样用户可以从 Python shell 中与之交互。 特别是，我们想要回答四个主要问题:
* Pytorch 是如何通过定义可以从 Python 代码中操作的 Tensor 类型来扩展 Python 解释器的？
* Pytorch 是如何封装实际定义 Tensor 的属性和方法的 c 库的？
* Pytorch 如何编写 Tensor 方法的代码？
* Pytorch 的构建系统如何将所有这些组件来编译和生成一个可行的应用程序的？

## 扩展 Python 解释器

PyTorch 定义了一个新的`torch`包。 在这篇文章中，我们将主要介绍 `._C` 模块。 这个模块被称为"扩展模块"——用 c 编写的 Python 模块。 这些模块允许我们定义新的内置对象类型(例如 `Tensor`) ，和调用 c/c++ 函数。 

`._C` 模块在 `torch/csrc/module.cpp` 中定义。 函数 `init_C()` / `PyInit__C()`函数创建`._C`模块并酌情添加方法定义。 `._C`模块被传递给一系列各不相同以 `__init()` 开头的函数，这些函数负责向模块中添加更多的对象，或者注册新的类型等。 

> 译注：关于`init_C()` / `PyInit__C()`这两个函数，前者在py2中被使用，后者在py3中被使用。

部分 `__init()` 函数如下：

```c
ASSERT_TRUE(THPDoubleTensor_init(module));
ASSERT_TRUE(THPFloatTensor_init(module));
ASSERT_TRUE(THPHalfTensor_init(module));
ASSERT_TRUE(THPLongTensor_init(module));
ASSERT_TRUE(THPIntTensor_init(module));
ASSERT_TRUE(THPShortTensor_init(module));
ASSERT_TRUE(THPCharTensor_init(module));
ASSERT_TRUE(THPByteTensor_init(module));
```

这些函数将不同类型的 `Tensor` 对象添加到 `_C` 模块，以便它们可以在模块中使用。 让我们来了解一下这些方法是如何工作的。

## The THPTensor Type

与底层的 TH 和 THC 库非常相似，PyTorch 定义了一个"通用的" Tensor 基类(实际上它的名字叫 THPTensor),这个基类用于生成许多不同的子类。 在考虑这个子类如何定义之前，让我们首先考虑如何在  Python 中定义一个新的内置类型，以及我们如何创建通用的 THPTensor 类。

Python 解释器将所有 Python 对象视为 `PyObject *` 类型的变量，它是所有 Python 对象的"基础类型"。 每个 Python 类型都包含对象的重新计数，以及指向对象类型对象的指针。 类型对象决定类型的属性。 例如，它可能包含与类型相关联的方法列表，并且调用哪个 c 函数来实现这些方法。 该对象还包含表示其状态所必需的任何字段。

```C
typedef struct {
    PyObject_HEAD
    double ob_fval;
} PyFloatObject;
```

```c
struct THPTensor {
    PyObject_HEAD
    THTensor *cdata;
};
```

```c
static PyTypeObject py_FloatType = {
    PyVarObject_HEAD_INIT(NULL, 0)
    "py.FloatObject",          /* tp_name */
    sizeof(PyFloatObject),     /* tp_basicsize */
    0,                         /* tp_itemsize */
    0,                         /* tp_dealloc */
    0,                         /* tp_print */
    0,                         /* tp_getattr */
    0,                         /* tp_setattr */
    0,                         /* tp_as_async */
    0,                         /* tp_repr */
    0,                         /* tp_as_number */
    0,                         /* tp_as_sequence */
    0,                         /* tp_as_mapping */
    0,                         /* tp_hash  */
    0,                         /* tp_call */
    0,                         /* tp_str */
    0,                         /* tp_getattro */
    0,                         /* tp_setattro */
    0,                         /* tp_as_buffer */
    Py_TPFLAGS_DEFAULT,        /* tp_flags */
    "A floating point number", /* tp_doc */
};
```

```c
PyTypeObject THPTensorType = {
  PyVarObject_HEAD_INIT(NULL, 0)
  ...
  THPTensor_(pynew), /* tp_new */
};
```

```c
static PyObject * THPTensor_(pynew)(PyTypeObject *type, PyObject *args, PyObject *kwargs)
{
  HANDLE_TH_ERRORS
  Py_ssize_t num_args = args ? PyTuple_Size(args) : 0;

  THPTensorPtr self = (THPTensor *)type->tp_alloc(type, 0);
// more code below
```

```python
x = torch.Tensor(10).fill_(1)
y = x[3] // y == 1
x[4] = 2
// etc.
```

```C
```

## Generic Builds

## PyTorch 封装